In [ ]:
Análise das palavras - glossários e sua representatividade no modelo

Problema:
    Estamos usando uma matriz (BOW) e estamos com muito problema de ter o match porém está fora do contexto.
        Ex1: "O cartão foi enviado" (viado - baixo-calão) - a apalavra deveria considerar um espaço no inicio;
        Ex2: "O cartão esta na bosta" (bosta - baixo-calão) - erro na transcrição "bolsa"
            
P: Qual a representatividade das palavras?

P: As palavras são efetivas?

P: Existe a possibilidade de alterar e qual o ganho?



ESTRATÉGIAS
0 - Escolha do primeiro alvo devido ao impacto na nota/score
    BAIXO_CALAO e FALTA DE ETICA BANCO

1 - Corrigindo o Termo : adicionando espaços, removendo palavras de baixa aderencia;
    Primeiro passo é identificar as palavras e sua frequencia;
    Fazer um comparativo com o uso de um segundo glossário com os ajustes necessários;
    
2 - Usando a Lemmatização;

3 - Avaliando a similaridade;

4 - LDA para explorar os tópicos e frequencias;


Plotar os resultados e inserir no PPT



Qual a solução ideal? 
Ter o match apenas nos registros dentro do contexto

Como eu que posso fazer isso com Data Science/ML?
Melhorar o BOW e criar uma solução de recomendação de contexto

Como essa solução será usada em produção? 
Abordagem com "ponto de corte" -> 
Ex: retornar apenas Top 3 Abordagem de ranking 
    -> ordene os vídeos mais interessantes primeiro Web app com os vídeos (links) e as previsões ordenadas

Como eu vou saber que deu certo?
Métrica primária: dos top N vídeos, quantos eu coloco na lista de Watch Later.
Métrica(s) secundária(s): quanto tempo eu passo selecionando vídeos



In [ ]:
http://localhost:8888/notebooks/_CURRICULUM/NPS/LDA_SK-NPS-Copy-9-RV.ipynb

In [6]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import openpyxl
pd.options.display.max_columns = None

#Dataviz
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns

#NLP
import re
import string
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from nltk.collocations import *
from collections import Counter
from nltk import bigrams
from nltk import trigrams
from nltk import ngrams

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from skmultilearn.adapt import MLkNN, MLARAM
from scipy.sparse import csr_matrix, lil_matrix

from skmultilearn.problem_transform import BinaryRelevance, ClassifierChain, LabelPowerset
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hamming_loss


import unicodedata

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinic\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
nlp = spacy.load('pt_core_news_sm')

In [ ]:
MAXI = [
    'conta agora',
    'tou vendo aqui'
]

MINI = [
    'temporario',
    'tou verificando aqui'
]

In [3]:
df = pd.read_excel('Base_reviews_eda.xlsx')

In [4]:
df

product_id           product_brand  overall_rating  \
0       113022329          philips walita               4   
1       113851581             roma jensen               4   
2       131788803                      lg               5   
3       122776350                ventisol               5   
4       126474851                  philco               5   
...           ...                     ...             ...   
38556   114653755  sulamericana fantasias               5   
38557   122665676           guabi petcare               5   
38558   119689669                     tes               5   
38559     6774907             marca facil               1   
38560   114081902                   kapos               1   

      recommend_to_a_friend reviewer_gender reviewer_state  \
0                       Yes               M             SP   
1                       Yes               F             SP   
2                       Yes               M             MG   
3                       Yes               M             SP   
4                       Yes               F             RJ   
...                     ...             ...            ...   
38556                   Yes               F             PR   
38557                   Yes               F             MS   
38558                   Yes               M             RS   
38559                    No               M             RJ   
38560                    No               F             ES   

                                                  connid  \
0      44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...   
1      ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...   
2      7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...   
3      a0fd1ad35b08d3b764ad6f884ef7183bf29fc7ebfcae65...   
4      74cfe73e7489f0ed1f4d1f012efbc5a8e0958c3559b65d...   
...                                                  ...   
38556  d954bac944fec6a0217570fbecf0381356cf17a49c10db...   
38557  68dc07b564b634ed6e42369a297be20afa25e728be0e8d...   
38558  e785a9b39043e77fd12c446ae3d5fded90ab95ad384faf...   
38559  e6fb0b19277d01c2a300c7837a105f3c369377e92f9c19...   
38560  ea9467aa73271fb4f68c04f4dd4f4eef304d6ee85441fb...   

                                           interactionid              dt_refe  \
0      44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...  01/04/2020 12:08:14   
1      ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...  01/04/2020 12:08:14   
2      7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...  01/04/2020 12:08:14   
3      a0fd1ad35b08d3b764ad6f884ef7183bf29fc7ebfcae65...  01/04/2020 12:08:14   
4      74cfe73e7489f0ed1f4d1f012efbc5a8e0958c3559b65d...  01/04/2020 12:08:14   
...                                                  ...                  ...   
38556  d954bac944fec6a0217570fbecf0381356cf17a49c10db...  30/03/2020 12:00:00   
38557  68dc07b564b634ed6e42369a297be20afa25e728be0e8d...  30/03/2020 12:00:00   
38558  e785a9b39043e77fd12c446ae3d5fded90ab95ad384faf...  30/03/2020 12:00:00   
38559  e6fb0b19277d01c2a300c7837a105f3c369377e92f9c19...  30/03/2020 12:00:00   
38560  ea9467aa73271fb4f68c04f4dd4f4eef304d6ee85441fb...  30/03/2020 12:00:00   

      data_abreviada  ...  QNT_RESP QNT_DETRATOR QNT_NEUTRO QNT_PROMOTOR  \
0         01/04/2020  ...         1            0          0            1   
1         01/04/2020  ...         1            1          0            0   
2         01/04/2020  ...         1            1          0            0   
3         01/04/2020  ...         1            0          0            1   
4         01/04/2020  ...         1            0          0            1   
...              ...  ...       ...          ...        ...          ...   
38556     30/03/2020  ...         1            0          0            1   
38557     30/03/2020  ...         1            0          0            1   
38558     30/03/2020  ...         1            1          0            0   
38559     30/03/2020  ...         1            1          0            0   

In [ ]:
LDA

In [7]:
evaluate = df['transcript'].astype(str)

count_vectorizer = TfidfVectorizer(stop_words=stopwords)

count_data = count_vectorizer.fit_transform(evaluate)

number_topics = 11
number_words = 5

TypeError: 'LazyCorpusLoader' object is not iterable

In [ ]:
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_index, topic in enumerate(model.components_):
        print(f"\'topic{topic_index}\':['"+"\',\'".join([words[i] for i in topic.argsort() [:-n_top_words -1:-1]]))
        
        
print('Topics found via LDA:')
print_topics(lda, count_vectorizer, number_words)

In [ ]:
# dicionario de topicos
topics = {'topic0':['networks' , 'image' , 'model' , 'galaxies' , 'properties' , 'materials' , 'population'],
          'topic1':['problems' , 'based' , 'pseudo' , 'time' , '2d' , 'deterministic' , 'graphene'],
          'topic2':['mathfrak' , 'algebra' , 'î¼' , 'quantum' , 'paper' , 'type' , 'hbar'],
          'topic3':['rm' , 'log' , 'mass' , 'rate' , 'star' , 'adaptive' , 'm_'],
          'topic4':['graphene' , 'electron' , 'angle' , 'dirac' , 'reid' , 'galaxies' , 'valley']}

In [ ]:
# Função que pega uma lista de palavras e um texto como entrada e conta quantas vezes as palavras da lista aparecem no texto
def count_words(words, text):
    counter = 0
    for word in words:
        for w in text.split():
            if word == w:
                counter+= 1
    return counter

In [ ]:
# Função para classificar cada resumo em um tópico, verificando o tópico que contém mais palavras.
# Para cada resumo, cria um dicionário {topic: qtd words da lista} e depois verifica qual é a chave no dicionário com o maior número, que será o número do tópico a ser adicionado a uma lista e devolvido.

def assign_topic(topic, text):
    assign = []
    for row in text:
        topic_counts = {}
        for key, value in topic.items():
            topic_counts[key] = count_words(value, row) #use function previously created
        assign.append(max(topic_counts, key=lambda k: topic_counts[k]))
    return assign

In [ ]:
# A lista de tópicos que irão no dataframe
topics_list = assign_topic(topics, nps_df.tx_coment)

# Adicionando a coluna de tópicos ao dataframe
nps_df['topics'] = topics_list

In [ ]:
plt.figure(figsize=(7,7))
labels = topics.items()
sizes = nps_df.topics.value_counts()
plt.pie(sizes, autopct='%1.f%%', startangle=90, textprops={'fontsize':14})
plt.legend(labels, loc='center left', bbox_to_anchor=(1,0,0.5,1))
plt.show()